<a href="https://colab.research.google.com/github/srihari-humbarwadi/ssd_tensorflow/blob/master/colab_tpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!rm -rf ssd_tensorflow
!git clone https://github.com/srihari-humbarwadi/ssd_tensorflow.git

import sys
from google.colab import auth
import tensorflow_gcs_config as tfgcs

auth.authenticate_user()

sys.path.append('ssd_tensorflow')

In [ ]:
from glob import glob
import logging
import os
from pprint import pprint

import tensorflow as tf

from ssd.common.callbacks import CallbackBuilder
from ssd.common.distribute import get_strategy
from ssd.common.config import load_config
from ssd.common.viz_utils import draw_boxes_cv2, imshow
from ssd.data.dataset_builder import DatasetBuilder
from ssd.losses.multibox_loss import MultiBoxLoss
from ssd.models.ssd_model import SSDModel

logger = tf.get_logger()
logger.setLevel(logging.INFO)

logger.info('version: {}'.format(tf.__version__))

In [ ]:
# !PYTHONPATH="ssd_tensorflow" python ssd_tensorflow/ssd/scripts/calculate_feature_shapes.py --image_height 1024 --image_width 1024 --num_feature_maps 7
# !PYTHONPATH="ssd_tensorflow" python ssd_tensorflow/ssd/scripts/calculate_scales.py -n 7 --s_first 0.04 --smin 0.06 --smax 0.98

In [ ]:
config = {
    'image_height': 512,
    'image_width': 512,
    'num_classes': 80,
    'classes':
    [
        "airplane",
        "apple",
        "backpack",
        "banana",
        "baseball bat",
        "baseball glove",
        "bear",
        "bed",
        "bench",
        "bicycle",
        "bird",
        "boat",
        "book",
        "bottle",
        "bowl",
        "broccoli",
        "bus",
        "cake",
        "car",
        "carrot",
        "cat",
        "cell phone",
        "chair",
        "clock",
        "couch",
        "cow",
        "cup",
        "dining table",
        "dog",
        "donut",
        "elephant",
        "fire hydrant",
        "fork",
        "frisbee",
        "giraffe",
        "hair drier",
        "handbag",
        "horse",
        "hot dog",
        "keyboard",
        "kite",
        "knife",
        "laptop",
        "microwave",
        "motorcycle",
        "mouse",
        "orange",
        "oven",
        "parking meter",
        "person",
        "pizza",
        "potted plant",
        "refrigerator",
        "remote",
        "sandwich",
        "scissors",
        "sheep",
        "sink",
        "skateboard",
        "skis",
        "snowboard",
        "spoon",
        "sports ball",
        "stop sign",
        "suitcase",
        "surfboard",
        "teddy bear",
        "tennis racket",
        "tie",
        "toaster",
        "toilet",
        "toothbrush",
        "traffic light",
        "train",
        "truck",
        "tv",
        "umbrella",
        "vase",
        "wine glass",
        "zebra",
    ],

    'clip_default_boxes': False,
    'scales': [0.04, 0.1, 0.26, 0.42, 0.58, 0.74, 0.9, 1.06],
    'feature_shapes': [[64, 64], [32, 32], [16, 16], [8, 8], [6, 6], [4, 4], [1, 1]],
    'aspect_ratios':
    [
        [0.5, 1, 2],
        [0.333, 0.5, 1, 2, 3],
        [0.333, 0.5, 1, 2, 3],
        [0.333, 0.5, 1, 2, 3],
        [0.5, 1, 2],
        [0.5, 1, 2],
        [0.5, 1, 2],
    ],
    'loc_variance': [0.1, 0.1, 0.2, 0.2],

    'match_iou_threshold': 0.5,
    'max_detections': 300,

    'nms_iou_threshold': 0.5,
    'score_threshold': 0.0001,

    'loc_loss_weight': 1.0,
    'cls_loss_weight': 2.0,
    'negatives_ratio': 3,
    'smooth_l1_delta': 1.0,

    'backbone': 'resnet_50_v2',
    'freeze_bn': False,

    'train_images': 117266,
    'val_images': 4952,

    'tfrecords_train': 'gs://tpu_datasets_us/coco_tfrecords/train*',
    'tfrecords_val': 'gs://tpu_datasets_us/coco_tfrecords/val*',

    'model_dir': 'gs://tpu_datasets_us/model_files/coco/resnet_50_512x512-8',

    'batch_size': 8,
    'lr_boundaries': [500, 120000, 180000],
    'lr_values': [0.0006, 0.001, 0.0001, 0.00002],
    'optimizer_momentum': 0.9,
    'nestrov': True,

    'l2_regularization': 0.0004,
    'epochs': 300,
    'patience': 300,
    'cache_dataset_in_memory': True,

    'resume_training': True,
    'clear_previous_runs': False,

    'use_mixed_precision': True,

    'use_gpu': False,
    'multi_gpu': False,

    'use_tpu': True,
    'tpu_name': 'grpc://' + os.environ['COLAB_TPU_ADDR'],

    'scale_lr': True,
    'scale_batch_size': True,

    'augment_val_dataset': False,
    'random_pad_to_square': True,
    'random_brightness': False,
    'random_contrast': True,
    'random_saturation': True,
    'random_flip_horizonal': False,
    'random_patch': True,

    'max_pad_ratio': 2,
    'brightness_max_delta': 0.2,
    'contrast_lower': 0.5,
    'contrast_upper': 1.5,
    'saturation_lower': 0.5,
    'saturation_upper': 1.5,
    'min_obj_covered': [0.3, 0.5, 0.7, 0.9],
    'area_range': [0.1, 1],
    'aspect_ratio_range': [0.667, 1.334],
}

In [ ]:
if config['use_mixed_precision']:
    if config['use_tpu']:
        dtype = 'mixed_bfloat16'
    elif config['use_gpu']:
        #         dtype = 'mixed_float16' # todo: implement loss scaling
        dtype = 'float32'
    else:
        dtype = 'float32'
else:
    dtype = 'float32'


policy = tf.keras.mixed_precision.experimental.Policy(dtype)
tf.keras.mixed_precision.experimental.set_policy(policy)

logger.info('Compute dtype: {}'.format(policy.compute_dtype))
logger.info('Variable dtype: {}'.format(policy.variable_dtype))

strategy = get_strategy(config)
tfgcs.configure_gcs_from_colab_auth()

epochs = config['epochs']

lr_values = list(config['lr_values'])
if config['scale_lr']:
    for i in range(len(lr_values)):
        lr_values[i] *= strategy.num_replicas_in_sync
config['lr_values'] = lr_values

batch_size = config['batch_size']
batch_size = batch_size if not config['scale_batch_size'] else batch_size * strategy.num_replicas_in_sync
config['batch_size'] = batch_size

train_steps = config['train_images'] // config['batch_size']
val_steps = config['val_images'] // config['batch_size']

In [ ]:
with strategy.scope():
    train_dataset = DatasetBuilder('train', config)
    val_dataset = DatasetBuilder('val', config)

    loss_fn = MultiBoxLoss(config)
    lr_sched = tf.optimizers.schedules.PiecewiseConstantDecay(config['lr_boundaries'], config['lr_values'])
    optimizer = tf.optimizers.SGD(lr_sched, momentum=config['optimizer_momentum'], nesterov=config['nestrov'])
    callbacks_list = CallbackBuilder('COCO_', config).get_callbacks()

    model = SSDModel(config)
    model.compile(loss_fn=loss_fn, optimizer=optimizer)
    if config['resume_training']:
        latest_checkpoint = tf.train.latest_checkpoint(os.path.join(config['model_dir'], 'checkpoints'))
        if latest_checkpoint is not None:
            logger.info('Loading weights from {}'.format(latest_checkpoint))
            model.load_weights(latest_checkpoint)
        else:
            logger.warning('No weights found, training from scratch')

In [ ]:
model.fit(train_dataset.dataset,
          epochs=epochs,
          steps_per_epoch=train_steps,
          validation_data=val_dataset.dataset,
          validation_steps=val_steps,
          callbacks=callbacks_list)

In [ ]:
from tqdm import tqdm 

for images, _ in val_dataset.dataset.take(1):
    for i in tqdm(range(images.shape[0])):
        image = tf.cast(images[i], dtype=policy.compute_dtype)
        detections = model.get_detections(image[None, ...])
        if 'resnet' in config['backbone']:
            image = image * 127.5 + 127.5
        categories = [config['classes'][cls_id] for cls_id in detections['cls_ids']]
        image = draw_boxes_cv2(image, detections['boxes'], categories)
        imshow(image, (8, 8))

In [ ]:
signatures = {
    'serving_default': model.get_detections.get_concrete_function(tf.TensorSpec([1, config['image_height'], config['image_width'], 3]))
}

In [ ]:
model.save('model_files/v1', save_format='tf', signatures=signatures)